一个Spark推荐系统引擎的实现
https://www.cnblogs.com/muchen/p/6882465.html

In [1]:
import os
import sys
spark_home = os.environ.get('SPARK_HOME',None)
print(spark_home)
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.10.4-src.zip'))

exec(open(os.path.join(spark_home,'python/pyspark/shell.py')).read())

/home/yang0/installer/spark-2.4.3-bin-hadoop2.7
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.3
      /_/

Using Python version 3.6.5 (default, Jul 11 2019 14:32:02)
SparkSession available as 'spark'.


In [14]:
# 载入评级数据
rawData = sc.textFile("./ml-100k/u.data")
# 展示一条记录
# 用户id | 影片id | 评分值 | 时间戳(timestamp格式)
rawData.first()  # '196\t242\t3\t881250949'

# 格式化数据集
rating_data = rawData.map(lambda line: line.split("\t"))

rating_data.first()  #['196', '242', '3', '881250949']

# 接下来需要将评分矩阵RDD转化为Rating格式的RDD

from pyspark.mllib.recommendation import Rating
# 将评分矩阵RDD中每行记录转换为Rating类型
# ratings = rating_data.map { case Array(user, movie, rating) => Rating(user.toInt, movie.toInt, rating.toDouble) }
ratings = rating_data.map(lambda v: Rating(int(v[0]), int(v[1]), float(v[2])))
ratings.first() # Rating(user=196, product=242, rating=3.0)

Rating(user=196, product=242, rating=3.0)

**训练推荐模型**

接下来可以进行ALS推荐系统模型训练了。MLlib中的ALS算法接收三个参数：

- rank：对应的是隐因子的个数，这个值设置越高越准，但是也会产生更多的计算量。一般将这个值设置为10-200；
- iterations：对应迭代次数，一般设置个10就够了；
- lambda：该参数控制正则化过程，其值越高，正则化程度就越深。一般设置为0.01。

1. 首先，执行以下代码，启动ALS训练：

In [15]:
# 导入ALS推荐系统算法包
from pyspark.mllib.recommendation import ALS

# 启动ALS矩阵分解
model = ALS.train(ratings, 50, 10, 0.01)


 这步将会使用ALS矩阵分解算法，对评分矩阵进行分解，且隐特征个数设置为50，迭代10次，正则化参数设为了0.01。

相对其他步骤，训练耗费的时间最多。

In [16]:
print(model)

返回类型为MatrixFactorizationModel对象，它将结果分别保存到两个(id,factor)RDD里面，分别名为userFeatures和productFeatures。

也就是评分矩阵分解后的两个子矩阵